# Text Classification

In [1]:
import csv
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier

import re
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from string import punctuation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from string import punctuation
from sklearn import svm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from nltk import ngrams
from itertools import chain
from wordcloud import WordCloud


In [3]:
odf = pd.read_csv('NEW.csv')
import csv
X= odf['Text']
Y=odf['Score']
data = {'Text':X, 'Score':Y}

In [4]:
df = pd.DataFrame(data) 
df.head()
#len(df)

,Text,Score
0,I have bought several of the Vitality canned d...,5.0
1,Product arrived labeled as Jumbo Salted Peanut...,1.0
2,This is a confection that has been around a fe...,4.0
3,If you are looking for the secret ingredient i...,2.0
4,Great taffy at a great price. There was a wid...,5.0


In [5]:
df=df.dropna()
df['Score']=df['Score'].astype(int)
df=df[df['Score']!=3]
len(df)
df['new_score']=-1
df['new_score']=[1 if x ==4 or x==5 else 0 for x in df['Score']] 
df.head()

,Text,Score,new_score
0,I have bought several of the Vitality canned d...,5,1
1,Product arrived labeled as Jumbo Salted Peanut...,1,0
2,This is a confection that has been around a fe...,4,1
3,If you are looking for the secret ingredient i...,2,0
4,Great taffy at a great price. There was a wid...,5,1


In [6]:
df['new_score'].value_counts()

1    38407
0     7581
Name: new_score, dtype: int64

In [7]:
X = df['Text']
y = df['new_score']


Feature Extraction: The models perform better with TfidfVectorizer than CountVectorizer.

In [8]:
vectorizer = TfidfVectorizer()

train_features = vectorizer.fit_transform(X)

In [9]:
#Naive Bayes Multinomial Classifier
nbm_clf = MultinomialNB()
%time nbm_clf.fit(train_features, y)
nbm_clf.class_count_

CPU times: user 51 ms, sys: 0 ns, total: 51 ms
Wall time: 48.9 ms


array([ 7581., 38407.])

In [10]:
#tested KNN with k 1 through 25 and k = 20 was the optimal one.
knn = KNeighborsClassifier(n_neighbors=20)
knn.fit(train_features,y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=20, p=2,
                     weights='uniform')

In [11]:
#Logistic Regression
Logi_clf = LogisticRegression()
%time Logi_clf.fit(train_features,y)

/home/shivi/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


CPU times: user 10.1 s, sys: 9.68 s, total: 19.7 s
Wall time: 3.44 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
estimators = [('mnb',nbm_clf),('lr',Logi_clf),('knn',knn)]

In [13]:
#Voting Classifier
voter_clf = VotingClassifier(estimators,voting='hard')
voter_clf.fit(train_features,y)

/home/shivi/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


VotingClassifier(estimators=[('mnb',
                              MultinomialNB(alpha=1.0, class_prior=None,
                                            fit_prior=True)),
                             ('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='warn', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('knn',
                              KNeighborsClassifier(algorithm='auto',
             

In [18]:
import pysbd
clean_text = df['Text']
#type(text)
sentences = []
for i in clean_text:
    seg = pysbd.Segmenter(language="en", clean=False)
    segmentedText = seg.segment(i)
    sentences.append(segmentedText)

As we can see with the skewed texts(2004 positive reviews and 208 negative), the models don't perform so well.

So let's see how the algorithms will perform on the data generated through Data Augmention

In [19]:
df['Text'] = df.apply(clean_text,axis = 1,pattern=pattern)
df['new_score'] = df.label.map({'negative': 1,'positive':0})
#skewed_data.head()

NameError: name 'pattern' is not defined

In [ ]:
#Models performance testing on out-of-sample data 
outSam_test = vectorizer.transform(out_of_sample['cleaned_txt'])
#change model name to see performance of different models
model_name = knn
pred = model_name.predict(outSam_test)
y_test = out_of_sample['to_binary']
print(f1_score(y_test,pred, average='macro')*100)
#out_of_sample[y_test<pred]